In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Đường dẫn đến geckodriver
gecko_path = r"C:\Users\HP\Downloads\geckodriver-v0.35.0-win64\geckodriver.exe"

# Cấu hình Firefox
options = Options()
service = Service(gecko_path)
driver = webdriver.Firefox(service=service, options=options)
wait = WebDriverWait(driver, 10)

# Mở trang web
url = "https://mogi.vn/ho-chi-minh/mua-can-ho?cp=3500"
driver.get(url)
time.sleep(3)

# Tạo file CSV
csv_filename = "mogi_data.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["STT", "Mã BĐS", "Tên dự án", "Địa chỉ", "Giá", "Diện tích", "Phòng ngủ", "Phòng tắm", "Pháp lý", "Ngày đăng", "Link BĐS"])

    stt = 0  # Số thứ tự tin đăng

    while True:  # Lặp qua tất cả các trang
        index = 1  # Reset index cho mỗi trang mới
        total_listings = len(driver.find_elements(By.XPATH, '//*[@id="property"]/div[1]/ul/li'))  # Đếm số tin có trên trang

        while index <= total_listings:  # Duyệt hết tin trên trang hiện tại
            try:
                listing_xpath = f'//*[@id="property"]/div[1]/ul/li[{index}]'
                listing_elements = driver.find_elements(By.XPATH, listing_xpath)
                if not listing_elements:
                    index += 1
                    continue

                listing = listing_elements[0]
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", listing)
                time.sleep(1)

                listing.click()
                time.sleep(1)

                property_link = driver.current_url
                
                def safe_get(xpath):
                    try:
                        return wait.until(EC.presence_of_element_located((By.XPATH, xpath))).text.strip()
                    except:
                        return ""
                
                project_name = safe_get('//*[@id="mogi-page-content"]/div[2]/div[1]/div[2]/div[1]/h1')
                address = safe_get('//*[@id="mogi-page-content"]/div[2]/div[1]/div[2]/div[2]')
                price = safe_get('//*[@id="mogi-page-content"]/div[2]/div[1]/div[2]/div[3]')
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(1)
                
                area = safe_get('//*[@id="mogi-page-content"]/div[2]/div[1]/div[2]/div[5]/div[1]/span[2]')
                bedrooms = safe_get('//*[@id="mogi-page-content"]/div[2]/div[1]/div[2]/div[5]/div[2]/span[2]')
                bathrooms = safe_get('//*[@id="mogi-page-content"]/div[2]/div[1]/div[2]/div[5]/div[3]/span[2]')
                legal = safe_get('//*[@id="mogi-page-content"]/div[2]/div[1]/div[2]/div[5]/div[4]/span[2]')
                ngaydang = safe_get('//*[@id="mogi-page-content"]/div[2]/div[1]/div[2]/div[5]/div[5]/span[2]')
                mabd = safe_get('//*[@id="mogi-page-content"]/div[2]/div[1]/div[2]/div[5]/div[6]/span[2]')
                
                stt += 1
                print(f"\nTin {stt}:")
                if mabd: print(f"  - Mã BĐS: {mabd}")
                if project_name: print(f"  - Tên dự án: {project_name}")
                if address: print(f"  - Địa chỉ: {address}")
                if price: print(f"  - Giá: {price}")
                if area: print(f"  - Diện tích: {area}")
                if bedrooms: print(f"  - Phòng ngủ: {bedrooms}")
                if bathrooms: print(f"  - Phòng tắm: {bathrooms}")
                if legal: print(f"  - Pháp lý: {legal}")
                if ngaydang: print(f"  - Ngày đăng: {ngaydang}")
                print(f"  - Link BĐS: {property_link}")

                writer.writerow([stt, mabd, project_name, address, price, area, bedrooms, bathrooms, legal, ngaydang, property_link])
                driver.back()
                time.sleep(1)

            except Exception as e:
                driver.back()
                time.sleep(1)
            
            index += 1

        try:
            next_page = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[3]/div[1]/div[2]/ul/li[10]/a')))
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_page)
            time.sleep(1)
            next_page.click()
            time.sleep(2)
        except:
            break  # Thoát vòng lặp nếu không có trang tiếp theo

# Đóng trình duyệt sau khi hoàn thành
driver.quit()
print("Đã hoàn thành thu thập dữ liệu.")



Tin 1:
  - Mã BĐS: 21978687
  - Tên dự án: 3PN giá rất tốt bán nhanh trước tết 3.07 tỷ, Skyline 83m2 Quận 7
  - Địa chỉ: Hoàng Quốc Việt, Phường Phú Thuận, Quận 7, TPHCM
  - Giá: 3 tỷ 50 triệu
  - Diện tích: 83 m2
  - Phòng ngủ: 3
  - Phòng tắm: 2
  - Pháp lý: Sổ hồng
  - Ngày đăng: 17/01/2023
  - Link BĐS: https://mogi.vn/quan-7/mua-can-ho-chung-cu/3pn-gia-rat-tot-ban-nhanh-truoc-tet-3-07-ty-skyline-83m2-quan-7-id21978687

Tin 2:
  - Mã BĐS: 21978700
  - Tên dự án: 2.05-2.1 tỷ 1PN chủ kẹt tiền cần chuyển nhượng nhanh giá tốt bao hết
  - Địa chỉ: Đào Trí, Phường Phú Thuận, Quận 7, TPHCM
  - Giá: 2 tỷ 50 triệu
  - Diện tích: 53 m2
  - Phòng ngủ: 1
  - Phòng tắm: 1
  - Pháp lý: Hợp đồng mua bán
  - Ngày đăng: 17/01/2023
  - Link BĐS: https://mogi.vn/quan-7/mua-can-ho-chung-cu/2-05-2-1-ty-1pn-chu-ket-tien-can-chuyen-nhuong-nhanh-gia-tot-bao-het-id21978700

Tin 3:
  - Mã BĐS: 21978689
  - Tên dự án: Sổ hồng bán nhanh 3.1 tỷ 3PN full nội thất Q7 giá tốt bán trước tết
  - Địa chỉ: Đào Trí, 

InvalidSessionIdException: Message: Tried to run command without establishing a connection
